In [1]:
import orjson

In [2]:
with open('eval_results/gorilla-openfunctions-v2-TIL24-r16-a16-ctx768-v2-5bit-hb6-3500samples_score.json', 'rb') as f:
    results = orjson.loads(f.read())
with open('eval_outputs/gorilla-openfunctions-v2-TIL24-r16-a16-ctx768-v2-5bit-hb6-3500samples.json', 'rb') as f:
    infer_output = orjson.loads(f.read())
with open('../../data/nlp.jsonl', 'rb') as f:
    label = [orjson.loads(line.strip()) for line in f if line.strip() != ""]

In [5]:
failed = {k: v for k, v in results.items() if v['target'] < 1 or v['tool'] < 1}
failed_heading = {k: v for k, v in results.items() if v['heading'] < 1}

In [8]:
# 230 Turret Foxtrot, engage the target at heading two seven five, red and yellow missile, deploy surface-to-air missiles. Turret Foxtrot, prepare for strike. It give [{'heading': '275', 'tool': 'red and yellow missile', 'target': 'deploy surface-to-air missiles'}]

In [7]:
failed

{'2739': {'heading': 1.0, 'target': 1.0, 'tool': 0.0}}

In [11]:
import orjson
with open('../../data/nlp.jsonl', 'rb') as f:
    label = [orjson.loads(line.strip()) for line in f if line.strip() != ""]

give_none_if_not_specified_string = ' Give None if not specified.'
functions = [
    {
        "name": "control_turret",
        "description": "Control the turret by giving it heading, tool to use and target description",
        "parameters": {
            "type": "object",
            "properties": {
                "heading": {"type": "string", "description": f"Heading of target in three arabic numbers and multiples of five (005 to 360).{give_none_if_not_specified_string}"},
                "tool": {"type": "string", "description": f"Tool to use or deploy.{give_none_if_not_specified_string}"},
                "target": {"type": "string", "description": f"Description of the target or enemy, exclude any quantifiers like 'the' or 'a'. It is a phrase that describe the appearance of the target like its color and type. Include ONLY the appearance and NOTHING else like its heading.{give_none_if_not_specified_string}"}
            },
            "required": ["heading", "tool", "target"],
        },
    }
]
functions_string = orjson.dumps(functions).decode('utf-8')
system_prompt = 'Convert the given instruction to turret into a function call to the control_turret function. Strictly ONLY one call is needed. If there are multiple instructions, use the FIRST one only.\n'

def format_data(example):
    user_query = example['transcript']
    heading, tool, target = example['heading'], example['tool'], example['target']
    return f'{system_prompt}### Instruction: <<function>>{functions_string}\n<<question>>{user_query}\n### Response: <<function>>control_turret(heading="{heading}",tool="{tool}",target="{target}")'

formatted_data = [format_data(example) for example in label]

In [13]:
with open('func_call.utf8', 'w+') as f:
    f.write('\n\n\n'.join(formatted_data))